In [2]:
"""

This module is responsible for reading outage email files from a folder
and using an LLM (like GPT-4) to extract structured information
such as partner name, outage duration, cause, etc.

The output is returned as a pandas DataFrame that can later be loaded
into DuckDB for analytics and reporting.

High-level workflow:
1. Read all `.txt` email files from a folder.
2. For each email, send the text to the LLM.
3. Ask the LLM to return JSON data with fixed fields.
4. Convert all outputs into a clean DataFrame.

"""

'\n\nThis module is responsible for reading outage email files from a folder\nand using an LLM (like GPT-4) to extract structured information\nsuch as partner name, outage duration, cause, etc.\n\nThe output is returned as a pandas DataFrame that can later be loaded\ninto DuckDB for analytics and reporting.\n\nHigh-level workflow:\n1. Read all `.txt` email files from a folder.\n2. For each email, send the text to the LLM.\n3. Ask the LLM to return JSON data with fixed fields.\n4. Convert all outputs into a clean DataFrame.\n\n'

In [3]:
import os
import json
import pandas as pd
from datetime import datetime
from langchain_openai import ChatOpenAI  # LangChain's OpenAI chat model interface
from dotenv import load_dotenv 
from pathlib import Path
load_dotenv(override=True)  # take environment variables from .env file


e:\AI Utils\Partner Performance Report\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [4]:
# -------------------------------
# Read environment variables
# -------------------------------
HITEC_EMAIL_DIR = os.getenv("GLOBAL_HITEC_EMAIL_DIR")
HITEC_EMAIL_JSON = os.getenv("GLOBAL_HITEC_EMAIL_JSON")
print(HITEC_EMAIL_DIR, HITEC_EMAIL_JSON)

Global_Hitec_Outage_Emails GLOBAL_HITEC_OUTAGE_JSON


In [5]:
# -------------------------------------------------------------------
# STEP 1: Create a helper function to initialize the LLM model
# -------------------------------------------------------------------
def get_llm(model_name="gpt-4o-mini"):
    """
    This function initializes the Large Language Model (LLM).
    In this case, we’re using the GPT-4 model provided through LangChain.

    - model_name: defines which OpenAI model to use.
    - temperature: controls randomness (0 = deterministic answers).

    Returns:
        llm object → can be used to call llm.predict(prompt)
    """
    
    return ChatOpenAI(model_name=model_name, temperature=0)

In [6]:
# -------------------------------------------------------------------
# STEP 2: Define the LLM prompt template
# -------------------------------------------------------------------
PARSER_PROMPT = """
You are an expert system that parses structured outage summary emails.
Your task is to extract outage-related details from the email below.

Extract the following fields and return ONLY valid JSON (no explanations):

- partner_name
- outage_type
- issue_details
- current_status
- business_impact
- manual_processing
- root_cause_available
- outage_start_time (ISO 8601 format)
- outage_end_time (ISO 8601 format)
- duration_hours (numeric)
- resolution_details
- email_subject
- email_date

Input Email:
-------------------
{email_text}
-------------------

Rules:
- Return only valid JSON.
- Convert all dates to ISO 8601 format (e.g., 2025-11-05T07:48:00-08:00).
- If data is missing, put null.
- If start and end times exist, calculate duration_hours.
"""




In [7]:


# -------------------------------------------------------------------
# STEP 3: Create a function to parse ONE email using LLM
# -------------------------------------------------------------------
from urllib import response


def parse_outage_email(llm, email_text: str):
    """
    Sends one email’s text to the LLM and returns a parsed JSON object.

    Parameters:
        llm        → The initialized LLM model object (from get_llm()).
        email_text → The raw text of one outage email.

    Returns:
        Dictionary containing all parsed fields + a timestamp.
    """

    # Prepare the LLM prompt by inserting the email text into the template
    prompt = PARSER_PROMPT.format(email_text=email_text)

    try:
        # Send the prompt to the LLM and capture its response
        response = llm.invoke(prompt)
        raw_response = response.content.strip()

        # The LLM might include extra text before/after JSON, so extract clean JSON only
        start = raw_response.find("{")
        end = raw_response.rfind("}")
        if start == -1 or end == -1:
            raise ValueError("No valid JSON object found in LLM response.")

        # Extract JSON substring
        json_str = raw_response[start:end+1]

        # Convert JSON string into Python dictionary
        parsed = json.loads(json_str)

    except Exception as e:
        # If something goes wrong (bad format, LLM failure, etc.)
        # we capture the error but still return a record (so processing continues)
        parsed = {
            "error": str(e),
            "raw_text": email_text[:2000]  # keep partial text for debugging
        }

    # Add a timestamp to track when this email was parsed
    parsed["parsed_at"] = datetime.utcnow().isoformat()

    # Return a Python dictionary of the parsed data
    return parsed



In [8]:

# -------------------------------------------------------------------
# STEP 4: Parse ALL emails from a folder
# -------------------------------------------------------------------
def parse_emails_from_folder(folder_path: str, llm):
    """
    Reads all text (.txt) email files from the given folder,
    uses the LLM to parse each one, and returns a pandas DataFrame.

    Parameters:
        folder_path → path to folder containing email files
        llm         → LLM object returned by get_llm()

    Returns:
        pandas.DataFrame containing one row per email with structured fields
    """
    #Create full directory path
    full_folder_path = Path(folder_path)
    #print("full folder path *********" + str(full_folder_path))
    # Create an empty list to hold results
    records = []

    # Loop through all files in the folder
    for file in os.listdir(full_folder_path):
        # Skip non-text files
        if not file.endswith(".txt"):
            continue

        # Full file path
        full_path = os.path.join(full_folder_path, file)
        #print("full file path *********" + str(full_path))

        # Read the email content from the file
        with open(full_path, "r", encoding="utf-8", errors="ignore") as f:
            email_text = f.read()

        #print(f"🔍 Parsing email file: {file} ...")

        # Parse one email using the LLM
        parsed = parse_outage_email(llm, email_text)

        # Keep track of the original file name for traceability
        parsed["source_file"] = file

        # Add parsed dictionary to list
        records.append(parsed)

    # Convert list of dictionaries into a pandas DataFrame
    df = pd.DataFrame(records)

    print(f"✅ Parsed {len(df)} emails successfully.")
    return df


In [9]:
"""
storage.py
-----------
This module handles storing the parsed outage email data
into a DuckDB database.

DuckDB is a lightweight, high-speed, SQL-based analytical database
that runs in a single file (like SQLite, but optimized for analytics).

The workflow:
1. Initialize a DuckDB connection and create a table (if not exists).
2. Insert parsed data (from a pandas DataFrame) into that table.
"""

import duckdb
import pandas as pd



# -------------------------------------------------------------------
# STEP 1: Define the database schema (table structure)
# -------------------------------------------------------------------
DB_SCHEMA = """
CREATE TABLE IF NOT EXISTS outages (
    partner_name VARCHAR,
    outage_type VARCHAR,
    issue_details VARCHAR,
    current_status VARCHAR,
    business_impact VARCHAR,
    manual_processing VARCHAR,
    root_cause_available VARCHAR,
    outage_start_time TIMESTAMP,
    outage_end_time TIMESTAMP,
    duration_hours DOUBLE,
    resolution_details VARCHAR,
    email_subject VARCHAR,
    email_date DATE,
    parsed_at TIMESTAMP,
    source_file VARCHAR
);
"""



# -------------------------------------------------------------------
# STEP 2: Function to initialize DuckDB
# -------------------------------------------------------------------
def init_db(in_memory=True):
    """
    Initializes the DuckDB database.

    Parameters:
        in_memory → if True, creates a temporary DB (for testing);
                     if False, saves data to a local file "outages.duckdb".

    Returns:
        Connection object → used to run SQL commands.
    """

    # Create a connection to the database
    if in_memory:
        con = duckdb.connect(database=":memory:")
        print("🧠 Using in-memory DuckDB (data will be lost after shutdown).")
    else:
        con = duckdb.connect(database="outages.duckdb")
        print("💾 Using persistent DuckDB file (outages.duckdb).")

    # Create table if it doesn’t exist yet
    con.execute(DB_SCHEMA)
    return con



# -------------------------------------------------------------------
# STEP 3: Function to load a DataFrame into DuckDB
# -------------------------------------------------------------------
def load_dataframe_to_duckdb(con, df: pd.DataFrame):
    """
    Inserts the parsed outage email data (from pandas DataFrame)
    into the 'outages' table in DuckDB.

    Parameters:
        con → active DuckDB connection
        df  → pandas DataFrame containing parsed email data
    """

    if df.empty:
        print("⚠️ No records found — nothing to insert.")
        return

    # Register the DataFrame as a temporary table in DuckDB
    con.register("df_temp", df)

    # Insert data into the main outages table using SQL
    con.execute("""
        INSERT INTO outages
        SELECT 
            partner_name,
            outage_type,
            issue_details,
            current_status,
            business_impact,
            manual_processing,
            root_cause_available,
            outage_start_time,
            outage_end_time,
            duration_hours,
            resolution_details,
            email_subject,
            email_date,
            parsed_at,
            source_file
        FROM df_temp
    """)

    # Unregister temporary table
    con.unregister("df_temp")

    #print(f"✅ {len(df)} records loaded into DuckDB successfully.")


In [1]:

# Step 1: Initialize the LLM
llm = get_llm()

# Step 2: Parse all emails from folder
df = parse_emails_from_folder(HITEC_EMAIL_DIR, llm)
#print(df.head())

# Step 3: Initialize database
con = init_db(in_memory=False)

#Delete existing data in outages table
con.execute("DELETE FROM outages")

# Step 4: Load parsed data into DuckDB
load_dataframe_to_duckdb(con, df)

# Step 5: Run a test SQL query
print("***********Running test query on outages table:******************")
print("Total outages:", con.execute("SELECT count(*) FROM outages").df())
#print(con.execute("SELECT * FROM outages").df())


NameError: name 'get_llm' is not defined